In [ ]:
### Imports
import datetime

import pytz

from MyFunctions.EMA import *
from IBridgePy.constants import OrderStatus
# from IBridgePy.quantopian import LimitOrder,TrailStopOrder
import csv
import os

os.chdir(r'D:\algo2\IBridgePy_Win_Anaconda38_64 - Copy')

from Utils import Algos
from Utils import EnterExits
from Utils import IndicatorCalculations
from Utils import Orders
from Utils import Plotters
from Utils import Utils

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import time

os.chdir(r'C:\Users\Lane Capital\OneDrive\Desktop\repos\IBridgePy_Win_Anaconda38_64 - Copy')
sys.path.append(os.path.join(os.getcwd(), '..'))

from trader_factory import build_active_IBridgePy_plus
from IBridgePy.IbridgepyTools import symbol
# from MyFunctions.EMA import EMA, EMA_1, MACD

sim = build_active_IBridgePy_plus('DU2628075') # Build an active trader to get connected to Interactive Brokers 

In [ ]:
sim.show_real_time_price(symbol('SPY') , 'ask_price')
# sim.request_historical_data(symbol('SPY'), '1 min', '3600 S') # Get historical data. Return is a pandas.DataFrame

In [ ]:
sim.request_historical_data(symbol('SPY'), '1 min', '86400 S') # Get historical data. Return is a pandas.DataFrame

In [ ]:
min1Data=sim.request_historical_data(symbol('SPY'), '1 min', '10800 S')
min5Data=sim.request_historical_data(symbol('SPY'), '5 mins', '10800 S')

In [ ]:
# load 1 and 5 min historical data for plotting EMA and candlestick charts

min5Path=r"C:\Users\Lane Capital\OneDrive\Desktop\repos\test_data\epoch_5min_data_2021-01-15.csv"
min1Path=r"C:\Users\Lane Capital\OneDrive\Desktop\repos\test_data\epoch_1min_data_2021-01-15.csv"
columnNames=["EpochTime", "open", "high", "low", "close", "volume"]
min1Data=pd.read_csv(min1Path, names=columnNames, header=0)
min5Data=pd.read_csv(min5Path, names=columnNames, header=0)


fakePrice=np.mean(min1Data.close.values) # get fake price for use in EMA

In [ ]:
# stuff to do today
# confirm data is in good format -check
# calc/plot all EMAs and look for crossings -check
# do a quick look at how algo would react
# do a daily plot for all enter/exits - check

In [ ]:
def calcEMA(min1Data, min5Data, price: float, nVec: list = [13, 48, 8]):
    """This function will take in your 1 minute data, 5 minute data (must both be dataFrames),and n vector 
    (vector containing periods for fast, slow, and Tline) and output a structure containing all relevant EMAs"""
    
    assert len(nVec)==3, "N vector must have 3 periods for fast, slow, and T line"
       
    # calculate all the EMAs
    
    fast = EMA(min1Data.close, price, nVec[0])
    fast_n = EMA(min1Data.close, price, nVec[0])
    fast_1 = EMA_1(min1Data.close, nVec[0])
    fast_2 = EMA_1(min1Data.close, nVec[0])

    slow = EMA(min1Data.close, price, nVec[1])
    slow_n = EMA(min1Data.close, price, nVec[1])
    slow_1 = EMA_1(min1Data.close, nVec[1])
    fast_slope = round((fast_n - fast_2)/2, 4)
    slow_slope = round((slow_n - slow_1) / 2, 4)

    t_line = EMA(min1Data.close, price, nVec[0])
    t_line_n = EMA(min1Data.close, price, nVec[2])
    t_line_1 = EMA_1(min1Data.close, nVec[2])
    t_slope = round((t_line_n-t_line_1) / 2, 4)
    
    fast_length_1 = EMA_1(min5Data.close, 12)
    slow_length_1 = EMA_1(min5Data.close, 26)
    fast_length = EMA_1(min5Data.close, 12)
    slow_length = EMA_1(min5Data.close, 26)

    macd_1 = fast_length_1 - slow_length_1
    macd = round(fast_length - slow_length, 2) # MACD at time T
    macd_slope = round((macd-macd_1)/2, 4)
    macd_9 = MACD(min5Data, macd, 9) # signal line
    
    # concatenate all outputs into an array before passing into a dataframe
    outputList = np.array([fast, fast_n, fast_1, fast_2, 
                    slow, slow_n, slow_1, fast_slope, slow_slope,
                    t_line, t_line_n, t_line_1, t_slope, 
                    fast_length_1, slow_length_1, fast_length, slow_length,
                    macd, macd_1, macd_slope, macd_9 ])
    
    # load all data into dataframe
    ema=pd.DataFrame(outputList.reshape(1,21), columns=["fast", "fast_n", "fast_1", "fast_2", 
                      "slow", "slow_n", "slow_1", "fast_slope", "slow_slope",
                      "t_line", "t_line_n", "t_line_1", "t_slope", 
                      "fast_length1", "slow_length_1", "fast_length", "slow_length",
                      "macd", "macd_1", "macd_slope", "macd_9" ])

    return ema
    
def processHistoricalData(min1Data, min5Data, nVec: list = [13, 48, 8]):
    """This function will take in your 1 minute data, 5 minute data (must both be dataFrames),and n vector 
    (vector containing periods for fast, slow, and Tline), loops through the close times, and calculates/stores
    EMA at relevant Time TODO clean this up to only calculate one time step"""
    
    maxPeriod=5*26*9 # max start periods for 5 minute data at 26 period filter (not quite sure about the 9 yet)
    price=[]
    time=[]
    
    # Initialize structure (use dataframe for now) # TODO make a class eventually
    ema=pd.DataFrame(columns=["fast", "fast_n", "fast_1", "fast_2", 
                      "slow", "slow_n", "slow_1", "fast_slope", "slow_slope",
                      "t_line", "t_line_n", "t_line_1", "t_slope", 
                      "fast_length1", "slow_length_1", "fast_length", "slow_length",
                      "macd", "macd_1", "macd_slope", "macd_9" ])


    # get time in minutes from data start
    min1Data.EpochTime=(min1Data.EpochTime-min1Data.EpochTime.iloc[0])/(60)
    min5Data.EpochTime=(min5Data.EpochTime-min5Data.EpochTime.iloc[0])/(60)

    #Start loop from maximum period to ensure each EMA function has time to spool up
    goodTime=min1Data.EpochTime[min1Data.EpochTime>maxPeriod] 
    goodTime=goodTime[:2335-390] # doing this because we are failing due to slightly less 5min data
    for t in goodTime: 
        i=np.argmax(t==min1Data.EpochTime) # get index of time vector
        # get data to put in EMA
        tempPrice=min1Data.close.iloc[i] # price at time t
        temp1Data=min1Data.iloc[:i] # historical values at time t

        # get 5 min data
        ind5=(np.argmax(min5Data.EpochTime>t)) # find index in 5 min data which is > than t
        temp5Price=min5Data.close.iloc[(ind5-1)] # get last 5 minute data price
        temp5Data=min5Data[:(ind5)] # get historical 5 minute values at time t

        # loop through each time step for 1 and 5 minute 
        temp = calcEMA(temp1Data, temp5Data, tempPrice, nVec=nVec) # calculate EMA
        # append dataframe with new row of data
        ema=ema.append(temp)
        price.append(tempPrice)
        time.append(i)
    ema.index=np.arange(len(ema.index))
    ema.insert(0,"time",time)
    ema.insert(1,"price",price)
    
    return ema

def plotDailyCandleStick(inputData, 
                 ema, 
                 emaList: list = ['fast', 'slow'], 
                 security: str = "SPY",
                 date: str = "01/15/2021",
                 saveFig: bool = True,
                 outPutFileName: str=r"C:\Users\Lane Capital\OneDrive\Desktop\repos\plots\TestCandle.png"):
    
    """This function takes in a historical input data dataframe, EMA dataframe from calcEMAVector 
    and plots the differnet EMAS against the price. Price is assumed to be the close price"""

    # TODO figure out how to get kwargs working for date and outputfilename
    # TODO get ylim as a function of data plotted
    # TODO get xlim as an input

    #create figure
    fig, ax = plt.subplots(figsize=(18,14))

    #define width of candlestick elements
    width = 0.5
    width2 = 0.05

    #define up and down inputData
    up = inputData[inputData.close>=inputData.open]
    down = inputData[inputData.close<inputData.open]

    #define colors to use
    col1 = 'green'
    col2 = 'red'

    for i,j in enumerate(emaList):
        exec('temp{}=ema.{}'.format(i,j))    
        exec('ax.plot(ema.time, temp{})'.format(i))


    #plot up inputData
    ax.bar(up.index,up.close-up.open,width,bottom=up.open,color=col1)
    ax.bar(up.index,up.high-up.close,width2,bottom=up.close,color=col1)
    ax.bar(up.index,up.low-up.open,width2,bottom=up.open,color=col1)

    #plot down inputData
    ax.bar(down.index,down.close-down.open,width,bottom=down.open,color=col2)
    ax.bar(down.index,down.high-down.open,width2,bottom=down.open,color=col2)
    ax.bar(down.index,down.low-down.close,width2,bottom=down.close,color=col2)

    ax.legend(emaList)
    ax.grid(which='major')
    ax.grid(which='minor', linewidth=0.25)
    ax.minorticks_on()
#     ax.set(xlabel="Minutes After Opening", ylabel="Stock Price (USD)", 
#        title=security+" Stock Price: "+date)

    ax.set(xlim=(400,500),ylim = (378,381),xlabel="Minutes After Opening", ylabel="Stock Price (USD)", 
           title=security+" Stock Price: "+date)

    if saveFig:
        fig.savefig(outPutFileName)
        
def plotDailyCandleStickMacd(inputData, 
                 ema, 
                 emaList: list = ['fast', 'slow', "macd"], 
                 security: str = "SPY",
                 date: str = "01/15/2021",
                 saveFig: bool = False,
                 outPutFileName: str=r"C:\Users\Lane Capital\OneDrive\Desktop\repos\plots\TestCandle.png"):
    
    """This function takes in a historical input data dataframe, EMA dataframe from calcEMAVector 
    and plots the differnet EMAS against the price. Price is assumed to be the close price"""

    # TODO figure out how to get kwargs working for date and outputfilename
    # TODO get ylim as a function of data plotted
    # TODO get xlim as an input

    #create figure
    fig, ax = plt.subplots(figsize=(18,14))

    #define width of candlestick elements
    width = 0.5
    width2 = 0.05

    #define up and down inputData
    up = inputData[inputData.close>=inputData.open]
    down = inputData[inputData.close<inputData.open]

    #define colors to use
    col1 = 'green'
    col2 = 'red'

    for i,j in enumerate(emaList):
        exec('temp{}=ema.{}'.format(i,j))    
        exec('ax.plot(ema.time, temp{})'.format(i))


    #plot up inputData
    ax.bar(up.index,up.close-up.open,width,bottom=up.open,color=col1)
    ax.bar(up.index,up.high-up.close,width2,bottom=up.close,color=col1)
    ax.bar(up.index,up.low-up.open,width2,bottom=up.open,color=col1)

    #plot down inputData
    ax.bar(down.index,down.close-down.open,width,bottom=down.open,color=col2)
    ax.bar(down.index,down.high-down.open,width2,bottom=down.open,color=col2)
    ax.bar(down.index,down.low-down.close,width2,bottom=down.close,color=col2)

    ax.legend(emaList)
    ax.grid(which='major')
    ax.grid(which='minor', linewidth=0.25)
    ax.minorticks_on()
    ax.set(xlabel="Minutes After Opening", ylabel="Stock Price (USD)", 
       title=security+" Stock Price: "+date)
    
    ax2=ax.twinx()
    ax2.plot(ema.time,ema.macd)

    if saveFig:
        fig.savefig(outPutFileName)
        
def EMA(values, current_price, n):
    start = len(values) - n+1
    sma_start = len(values) - (2*n) + 1
    ema = (sum(values[sma_start:start]))/n
    # print(ema)
    j = 1
    for i in range(start+1, len(values)):
        ema = (values[i] - ema) * (2 / (n + 1)) + ema
        # print(values[i])
        j += 1
    # print("\n")
    ema = (current_price - ema) * (2 / (n + 1)) + ema
    ema = round(ema, 2)
    return ema

def EMA_1(values, n):
    start = len(values) - n
    sma_start = (len(values) - (2*n))
    ema = (sum(values[sma_start:start]))/n
    v = values[sma_start:start]
    j = 1
    r = range(start+1, len(values))

    for i in range(start+1, len(values)):
        v2 = values[i]
        ema = (values[i] - ema) * (2 / (n + 1)) + ema
        j += 1
    ema = round(ema, 2)
    return ema

def MACD(values, current_macd, n):
    # values_list = [values[:-9].close, values[:-8].close, values[:-7].close, values[:-6].close,
    #                values[:-5].close, values[:-4].close, values[:-3].close, values[:-2].close, values[:-1].close]
    values_list = []
    for i in range(-18, 0):
        values_list.append(values[:i].close)
    macd_list = []
    for x in values_list:
        ###fast_length
        ema_12 = EMA_1(x, 12)
        ###slow_length
        start_26 = len(x) - 26
        ema_26 = EMA_1(x, 26)
        macd_x = ema_12 - ema_26
        macd_list.append(macd_x)

    macd_n = EMA(macd_list, current_macd, n)
    # start = len(macd_list) - n
    # macd_n = sum(macd_list[start:len(macd_list)]) / n
    # j = 1
    # for i in range(start, len(macd_list)):
    #     macd_n = (macd_list[i] - macd_n) * (9 / (j + 1)) + macd_n
    #     j+=1
    # macd_n = (current_macd - macd_n) * (9 / (n + 1)) + macd_n
    macd_n = round(macd_n, 2)
    return macd_n

def checkLong(context, ema, price):
    """This function takes in the input context, which is an IBPY standard structure and determines 
    whether or not to enter long, conditions for entering long are:
        - Price > Fast EMA > Slow EMA
    This function also has safeguards in place to reduce enters while in "the chop", these conditions are:
        - Fast is increasing
        -  macd > signal line
        This function returns a flag which determines whether or not to enter"""
    
    enter_flag = False # set enter flag to false Automatically
    print('checking long\n')
    print(price, ema.fast.values[0], ema.slow.values[0])
    if price > ema.fast.values[0] and ema.fast.values[0]> ema.slow.values[0]: # check to see we are in enter long conditions
        print('Enter Long before the chop\n')
        print('price, fast, last fast, slow, macd, macd_9 \n')
        print(price, ema.fast.values[0], ema.fast_1.values[0], ema.slow.values[0],ema.macd.values[0],ema.macd_9.values[0])
        if (ema.fast.values[0]  > ema.fast_1.values[0] ) and ema.macd.values[0] > ema.macd_9.values[0]: # checks to ensure we are not buying in the chop
            print('We passed the chop, buy long')
            enter_flag = True # buy               
    
    return enter_flag


def checkShort(context, ema, price):
    """This function takes in the input context, which is an IBPY standard structure and determines 
    whether or not to enter short, conditions for entering short are:
        - Price < Fast EMA < Slow EMA
    This function also has safeguards in place to reduce enters while in "the chop", these conditions are:
        - Fast is decreasing
        - macd is less than signal line
        This function returns a flag which determines whether or not to enter"""
    
    enter_flag = False  # set enter flag to false Automatically
    print('checking short\n')
    print(price, ema.fast.values[0], ema.slow.values[0])
    if price < ema.fast.values[0] and ema.fast.values[0]< ema.slow.values[0]: # check to see we are in enter long conditions
        print('Enter short before the chop\n')
        print('price, fast, last fast, slow, macd, macd_9 \n')
        print(price, ema.fast.values[0], ema.fast_1.values[0], ema.slow.values[0],ema.macd.values[0],ema.macd_9.values[0])
        if ema.fast.values[0] < ema.fast_1.values[0]  and ema.macd.values[0] < ema.macd_9.values[0] : # checks to ensure we are not buying in the chop
            print('We passed the chop, buy short')
            enter_flag = True # buy         
    return enter_flag
def checkEnter(context, ema, price, longBool, shortBool):
    """This function takes in the input context, which is an IBPY standard structure, and an EMA dataframe and 
        calls the checks if we are deciding to enter short or long"""
    
    # intialize Boolean
    enter_flag=False # set enter flag to false Automatically
    
    if checkShort(context,ema,price):# if we enter on either short or long, set flag to true
        enter_flag=True
        shortBool=True
    if checkLong(context,ema,price): # if we enter on either short or long, set flag to true
        enter_flag=True
        longBool=True
        
    return enter_flag, longBool, shortBool

def checkExit(context, ema, price, longBool, shortBool):
    """This function takes in the input context, which is an IBPY standard structure, and an EMA dataframe and 
        calls the checks if we are deciding to enter short or long"""
    
    exit_flag=False # set enter flag to false Automatically
    if sellShort(context,ema,price) and shortBool:
        exit_flag=True
        shortBool=False
    if sellLong(context,ema,price) and longBool: # if we enter on either short or long, set flag to true
        exit_flag=True
        longBool=False
    return exit_flag, longBool, shortBool

def sellLong(context, ema, price):
    """This function sells a long security when the following occurs
        - price closes below the fast EMA"""
    print('checking to sell long \n')
    print(price, ema.fast.values[0])
    exit_flag=False # set enter flag to false Automatically
    if price < ema.fast.values[0]: # check to if price is less than the fast
        print("Exit long Position")
        print('price, fast, last fast, slow, macd, macd_9 \n')
        print(price, ema.fast.values[0], ema.fast_1.values[0], ema.slow.values[0],ema.macd.values[0],ema.macd_9.values[0])
        exit_flag = True
    return exit_flag
    
def sellShort(context, ema, price):
    """This function sells a short security when the following occurs
        - price closes above the fast EMA"""
    print('checking to sell long \n')
    print(price, ema.fast.values[0])
    exit_flag=False # set enter flag to false Automatically
    if price > ema.fast.values[0]: # check to if price is greater than the fast
        print("Exit short Position")
        print('price, fast, last fast, slow, macd, macd_9 \n')
        print(price, ema.fast.values[0], ema.fast_1.values[0], ema.slow.values[0],ema.macd.values[0],ema.macd_9.values[0])
        exit_flag = True
    return exit_flag


# def doubleLong():
#     return

# def doubleShort():
#     return



In [ ]:
# RUN STUFF    
min5Path=r"C:\Users\Lane Capital\OneDrive\Desktop\repos\test_data\epoch_5min_data_2021-01-15.csv"
min1Path=r"C:\Users\Lane Capital\OneDrive\Desktop\repos\test_data\epoch_1min_data_2021-01-15.csv"
columnNames=["EpochTime", "open", "high", "low", "close", "volume"]
min1Data=pd.read_csv(min1Path, names=columnNames, header=0)
min5Data=pd.read_csv(min5Path, names=columnNames, header=0)


fakePrice=np.mean(min1Data.close.values) # get fake price for use in EMA    
ema=processHistoricalData(min1Data, min5Data)
plotDailyCandleStick(min1Data,ema) 

In [ ]:
plt.plot(ema.fast)
plt.plot(ema.slow)

In [ ]:
def EMA(values, current_price, n):
    start = len(values) - n+1
    sma_start = len(values) - (2*n) + 1
    ema = (sum(values[sma_start:start]))/n
    # print(ema)
    j = 1
    for i in range(start+1, len(values)):
        ema = (values[i] - ema) * (2 / (n + 1)) + ema
        # print(values[i])
        j += 1
    # print("\n")
    ema = (current_price - ema) * (2 / (n + 1)) + ema
    ema = round(ema, 2)
    return ema

def EMA_1(values, n):
    start = len(values) - n
    sma_start = (len(values) - (2*n))
    ema = (sum(values[sma_start:start]))/n
    v = values[sma_start:start]
    j = 1
    r = range(start+1, len(values))

    for i in range(start+1, len(values)):
        v2 = values[i]
        ema = (values[i] - ema) * (2 / (n + 1)) + ema
        j += 1
    ema = round(ema, 2)
    return ema

def MACD(values, current_macd, n):
    # values_list = [values[:-9].close, values[:-8].close, values[:-7].close, values[:-6].close,
    #                values[:-5].close, values[:-4].close, values[:-3].close, values[:-2].close, values[:-1].close]
    values_list = []
    for i in range(-18, 0):
        values_list.append(values[:i].close)
    macd_list = []
    for x in values_list:
        ###fast_length
        ema_12 = EMA_1(x, 12)
        ###slow_length
        start_26 = len(x) - 26
        ema_26 = EMA_1(x, 26)
        macd_x = ema_12 - ema_26
        macd_list.append(macd_x)

    macd_n = EMA(macd_list, current_macd, n)
    # start = len(macd_list) - n
    # macd_n = sum(macd_list[start:len(macd_list)]) / n
    # j = 1
    # for i in range(start, len(macd_list)):
    #     macd_n = (macd_list[i] - macd_n) * (9 / (j + 1)) + macd_n
    #     j+=1
    # macd_n = (current_macd - macd_n) * (9 / (n + 1)) + macd_n
    macd_n = round(macd_n, 2)
    return macd_n

In [ ]:
 ##Check if in position
        if context.order is None: # does this mean that we dont have stock currently?
            ###Long entry conditions
            if (fast > fast_1) and (slow_slope >= 0 or macd > macd_9): # make sure the space between fast/slow is increasing AND macd > signal line
                if (fast_slope >= (slow_slope + 0.02) and t_slope >= (fast_slope + 0.015)) or (t_slope >= 0.03): # TODO fix these hard coded contants
                    if context.price_1 > t_line >= price: # if price is over T line, buy
                        context.enter_flag = True # buy
                else: 
                    if (fast > slow) and context.price_1 > fast >= price:
                        context.enter_flag = True
                if context.enter_flag:
                    if context.entry_flag:
                        print("already in position")
                    elif not context.entry_flag:
                        print("enter long")
                        context.entry_price = price_mid
                        context.shares = int(context.value / price_mid)
                        context.ts_price_1 = price_mid - context.trail_1
                        context.ts_price_2 = price_mid - context.trail_2
                        context.order = order(context.security, context.shares, LimitOrder(price_mid),
                                              accountCode='default')
                        if get_order_status(context.order) in [OrderStatus.FILLED, OrderStatus.PRESUBMITTED,
                                                               OrderStatus.SUBMITTED]:
                            context.entry_time = sTime
                            context.entry_flag = True
                            context.long_flag = True
                            with open(file_name, 'a', newline='') as csvfile:
                                fieldnames = [
                                    "Time", "Entry Type", "Price", "Fast(EMA(13))", "Slow(EMA(48))",
                                    "Fast_Length(12)", "Slow_Length(26)", "MACD", "MACD(9)"
                                ]
                                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                                writer.writerow({'Time': sTime, 'Entry Type': "Long",
                                                 'Price': price, 'Fast(EMA(13))': fast, 'Slow(EMA(48))': slow,
                                                 'Fast_Length(12)': fast_length, 'Slow_Length(26)': slow_length,
                                                 "MACD": macd, 'MACD(9)': macd_9})
                            print('entry_flag: ', context.entry_flag, "\n", 'long_flag: ', context.long_flag)
            ###Short entry conditions
            elif (fast < fast_1) and (slow_slope >= 0 or macd < macd_slope):
                # (slow_slope <= 0)
                if (fast_slope <= (slow_slope - 0.02) and t_slope <= (fast_slope - 0.015)) or (t_slope <= -0.3):
                    if context.price_1 < t_line <= price:
                        context.enter_flag = True
                else:
                    if (fast < slow) and context.price_1 < fast <= price:
                        context.enter_flag = True
                if context.enter_flag:
                    if context.entry_flag:
                        print("already in position")
                    elif not context.entry_flag:
                        print("enter short")
                        context.entry_price = price_mid
                        value = - context.value
                        context.shares = int(value / price_mid)
                        context.ts_price_1 = price_mid + context.trail_1
                        context.ts_price_2 = price_mid + context.trail_2
                        context.order = order(context.security, context.shares, LimitOrder(price_mid),
                                              accountCode='default')
                        if get_order_status(context.order) in [OrderStatus.FILLED, OrderStatus.PRESUBMITTED,
                                                               OrderStatus.SUBMITTED]:
                            context.entry_time = sTime
                            context.entry_flag = True
                            context.short_flag = True
                            with open(file_name, 'a', newline='') as csvfile:
                                fieldnames = [
                                    "Time", "Entry Type", "Price", "Fast(EMA(13))", "Slow(EMA(48))",
                                    "Fast_Length(12)", "Slow_Length(26)", "MACD", "MACD(9)"
                                ]
                                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                                writer.writerow({'Time': sTime, 'Entry Type': "Short",
                                                 'Price': price, 'Fast(EMA(13))': fast, 'Slow(EMA(48))': slow,
                                                 'Fast_Length(12)': fast_length, 'Slow_Length(26)': slow_length,
                                                 "MACD": macd, 'MACD(9)': macd_9})
                            print('entry_flag: ', context.entry_flag, "\n", 'short_flag: ', context.short_flag)

In [ ]:
## Base Algorithm

# we dont own security:
    # trigger point: fast and slow cross:
        # fast>slow:
            # enter (long) on next candle (with code to ensure that current EMA >= previous EMA)
            # hold until lines cross and fast<slow
        # fast<slow:
            # short
    
# we do own security
    # trigger point
        # fast>slow
            # hold
        # fast<slow
            # sell and short

In [ ]:
# we do own security
    # trigger point
        # close price<fast
            # sell
            
# After we sell
# price closes above fast

# We are basically looking for price to close between fast/slow line

In [ ]:
#  Go Long when:
    # - lines cross, fast>slow (only first time)
    # - price>fast>slow
# go short when:
    # price<fast<slow
    
# close a position when:
    # Long - price closes below fast
    # short - price closes above fast

In [ ]:
def checkLong(context, ema, price):
    """This function takes in the input context, which is an IBPY standard structure and determines 
    whether or not to enter long, conditions for entering long are:
        - Price > Fast EMA > Slow EMA
    This function also has safeguards in place to reduce enters while in "the chop", these conditions are:
        - Fast is increasing
        - slow is increasing
        -  macd > signal line
        This function returns a flag which determines whether or not to enter"""
    
    enter_flag = False # set enter flag to false Automatically
    print('checking long\n')
    print(price, ema.fast.values[0], ema.slow.values[0])
    if price > ema.fast.values[0] and ema.fast.values[0]> ema.slow.values[0]: # check to see we are in enter long conditions
        print('Enter Long before the chop\n')
        print('price, fast, last fast, slow, macd, macd_9 \n')
        print(price, ema.fast.values[0], ema.fast_1.values[0], ema.slow.values[0],ema.macd.values[0],ema.macd_9.values[0])
        if (ema.fast.values[0]  > ema.fast_1.values[0] ) and (ema.slow_slope.values[0]  >= 0 or ema.macd.values[0]  > ema.macd_9.values[0] ): # checks to ensure we are not buying in the chop
            print('We passed the chop, buy long')
            enter_flag = True # buy               
    
    return enter_flag


def checkShort(context, ema, price):
    """This function takes in the input context, which is an IBPY standard structure and determines 
    whether or not to enter short, conditions for entering short are:
        - Price < Fast EMA < Slow EMA
    This function also has safeguards in place to reduce enters while in "the chop", these conditions are:
        - Fast is decreasing
        - slow is decreasing
        - macd is decreasing
        This function returns a flag which determines whether or not to enter"""
    
    enter_flag = False  # set enter flag to false Automatically
    print('checking short\n')
    print(price, ema.fast.values[0], ema.slow.values[0])
    if price < ema.fast.values[0] and ema.fast.values[0]< ema.slow.values[0]: # check to see we are in enter long conditions
        print('Enter short before the chop\n')
        print('price, fast, last fast, slow, macd, macd_9 \n')
        print(price, ema.fast.values[0], ema.fast_1.values[0], ema.slow.values[0],ema.macd.values[0],ema.macd_9.values[0])
        if (ema.fast.values[0]  < ema.fast_1.values[0] ) and (ema.slow_slope.values[0] >= 0 or ema.macd.values[0] < ema.macd_slope.values[0]): # checks to ensure we are not buying in the chop
            print('We passed the chop, buy short')
            enter_flag = True # buy         
    return enter_flag

def checkEnter(context, ema, price, longBool, shortBool):
    """This function takes in the input context, which is an IBPY standard structure, and an EMA dataframe and 
        calls the checks if we are deciding to enter short or long"""
    
    # intialize Boolean
    enter_flag=False # set enter flag to false Automatically
    
    if checkShort(context,ema,price):# if we enter on either short or long, set flag to true
        enter_flag=True
        shortBool=True
    if checkLong(context,ema,price): # if we enter on either short or long, set flag to true
        enter_flag=True
        longBool=True
        
    return enter_flag, longBool, shortBool

def checkExit(context, ema, price, longBool, shortBool):
    """This function takes in the input context, which is an IBPY standard structure, and an EMA dataframe and 
        calls the checks if we are deciding to enter short or long"""
    
    exit_flag=False # set enter flag to false Automatically
    if sellShort(context,ema,price) and shortBool:
        exit_flag=True
        shortBool=False
    if sellLong(context,ema,price) and longBool: # if we enter on either short or long, set flag to true
        exit_flag=True
        longBool=False
    return exit_flag, longBool, shortBool

def sellLong(context, ema, price):
    """This function sells a long security when the following occurs
        - price closes below the fast EMA"""
    print('checking to sell long \n')
    print(price, ema.fast.values[0])
    exit_flag=False # set enter flag to false Automatically
    if price < ema.fast.values[0]: # check to if price is less than the fast
        print("Exit long Position")
        print('price, fast, last fast, slow, macd, macd_9 \n')
        print(price, ema.fast.values[0], ema.fast_1.values[0], ema.slow.values[0],ema.macd.values[0],ema.macd_9.values[0])
        exit_flag = True
    return exit_flag
    
def sellShort(context, ema, price):
    """This function sells a short security when the following occurs
        - price closes above the fast EMA"""
    print('checking to sell long \n')
    print(price, ema.fast.values[0])
    exit_flag=False # set enter flag to false Automatically
    if price > ema.fast.values[0]: # check to if price is greater than the fast
        print("Exit short Position")
        print('price, fast, last fast, slow, macd, macd_9 \n')
        print(price, ema.fast.values[0], ema.fast_1.values[0], ema.slow.values[0],ema.macd.values[0],ema.macd_9.values[0])
        exit_flag = True
    return exit_flag


# def doubleLong():
#     return

# def doubleShort():
#     return



In [ ]:
def testAlgoBasic():
    initialPrice=t.show_real_time_price(symbol('SPY') , 'ask_price')
    enterFlag=False
    exitFlag=False
    numTimeSteps=60*24 # 1 minute data for one day, this number is used to initialize an output array for later storage
    data=np.zeros((numTimeSteps,25)) # array where the first 21 elements are all the EMAs, then price, enter, exit and time
    longBool=False # flag to determine if we own long stock
    ShortBool=False # flag to determine if we own short stock
    i=0
    context=pd.DataFrame(np.array([initialPrice,enterFlag,exitFlag]).reshape(1,3),columns=["price","enterFlag","exitFlag"])
    for i in np.linspace(0,500,1):#range(data.shape[0]):
        print(i)
        tic=time.time()
        # get historical data
        min1Data=t.request_historical_data(symbol('SPY'), '1 secs', '2000 S')
        min5Data=t.request_historical_data(symbol('SPY'), '5 secs', '2000 S')
        price=t.show_real_time_price(symbol('SPY') , 'ask_price')

        timeCurrent=time.time() # get current time for logging

        emaVector=calcEMA(min1Data, min5Data, price) # calculate EMA

        enterFlag=checkEnter(context,emaVector,price) # check if we want to enter

        exitFlag=checkExit(context,emaVector,price)# if we have security, check to exit

        # take all inputs and load them into the output array
        data[i,:]=np.concatenate((emaVector.values[0].reshape(1,21),np.array([price,enterFlag,exitFlag,timeCurrent]).reshape(1,4)),axis=1)

        toc=time.time() # record time to calculate sleep time
        print('Code took {} seconds'.format(toc-tic))
        return data
#         if toc-tic <1:
#             time.sleep(1-(toc-tic)) # sleep until loop takes one second to run
#         else:
#             break
# save exit/entry points and their associated indicators
# save all EMA, enter/exit flag, price

In [ ]:
def algoBase(context, ema, price, longBool, shortBool):
    """This function takes in  a price, ema, and security and checks if we enter/exit 
    at the current time step, and outputs an enter and exit flag"""
    
    # TODO check to buy even if we own
    # TODO check for the chop
    # TODO fix flag stuff
    # TODO validate algorithm entry/exit points
    
    enterFlag=False
    exitFlag=False
    # if we dont own stock, check to enter
    if not longBool and not shortBool: 
        enterFlag,longBool,shortBool =checkEnter(context,ema,price,longBool,shortBool) # check if we want to enter
    else: # if we own, check to exit
        exitFlag,longBool,shortBool=checkExit(context,ema,price,longBool,shortBool)# if we have security, check to exit
    
    return enterFlag, exitFlag,longBool,shortBool
    
    

In [ ]:
testData=testAlgoBasic()

In [ ]:
def testPlotter(inputData, 
                 ema, 
                 emaList: list = ['fast', 'slow', "macd"], 
                 security: str = "SPY",
                 date: str = "01/15/2021",
                 saveFig: bool = False,
                 outPutFileName: str=r"C:\Users\Lane Capital\OneDrive\Desktop\repos\plots\TestCandle.png"):
    
    """This function takes in a historical input data dataframe, EMA dataframe from calcEMAVector 
    and plots the differnet EMAS against the price. Price is assumed to be the close price"""

    # TODO figure out how to get kwargs working for date and outputfilename
    # TODO get ylim as a function of data plotted
    # TODO get xlim as an input

    #create figure
    fig, ax = plt.subplots(figsize=(18,14))

    #define width of candlestick elements
    width = 0.5
    width2 = 0.05

    #define up and down inputData
    up = inputData[inputData.close>=inputData.open]
    down = inputData[inputData.close<inputData.open]

    #define colors to use
    col1 = 'green'
    col2 = 'red'

    for i,j in enumerate(emaList):
        exec('temp{}=ema.{}'.format(i,j))    
        exec('ax.plot(ema.time, temp{})'.format(i))


    #plot up inputData
    ax.bar(up.index,up.close-up.open,width,bottom=up.open,color=col1)
    ax.bar(up.index,up.high-up.close,width2,bottom=up.close,color=col1)
    ax.bar(up.index,up.low-up.open,width2,bottom=up.open,color=col1)

    #plot down inputData
    ax.bar(down.index,down.close-down.open,width,bottom=down.open,color=col2)
    ax.bar(down.index,down.high-down.open,width2,bottom=down.open,color=col2)
    ax.bar(down.index,down.low-down.close,width2,bottom=down.close,color=col2)

    ax.legend(emaList)
    ax.grid(which='major')
    ax.grid(which='minor', linewidth=0.25)
    ax.minorticks_on()
    ax.set(xlabel="Minutes After Opening", ylabel="Stock Price (USD)", 
       title=security+" Stock Price: "+date)
    
    ax2=ax.twinx()
    ax2.plot(ema.time,ema.macd)

    if saveFig:
        fig.savefig(outPutFileName)

In [ ]:
# def algoHistoricalData(min1Data, min5Data, nVec: list = [13, 48, 8]):
nVec: list = [13, 48, 8]
min1Data=sim.request_historical_data(symbol('SPY'), '1 min', '86400 S')
min5Data=sim.request_historical_data(symbol('SPY'), '5 mins', '86400 S')
initialPrice=sim.show_real_time_price(symbol('SPY') , 'ask_price')
maxPeriod=5*26*9 # max start periods for 5 minute data at 26 period filter (not quite sure about the 9 yet, it just works)
price=[]
time=[]
enterFlag=False
exitFlag=False
shortBool=False
longBool=False
numTimeSteps=60*24 # 1 minute data for one day, this number is used to initialize an output array for later storage
data=np.zeros((min1Data.index.shape[0],25)) # array where the first 21 elements are all the EMAs, then price, enter, exit and time
i=0
z=0
# Initialize structures used later (use dataframe for now) # TODO make a class eventually
context=pd.DataFrame(np.array([initialPrice,enterFlag,exitFlag]).reshape(1,3),columns=["price","enterFlag","exitFlag"])

ema=pd.DataFrame(columns=["fast", "fast_n", "fast_1", "fast_2", 
                  "slow", "slow_n", "slow_1", "fast_slope", "slow_slope",
                  "t_line", "t_line_n", "t_line_1", "t_slope", 
                  "fast_length1", "slow_length_1", "fast_length", "slow_length",
                  "macd", "macd_1", "macd_slope", "macd_9" ])


# get time in minutes from data start
min1Data['EpochTime']=[(min1Data.index[k].timestamp()-min1Data.index[0].timestamp())/(60) for k in range(len(min1Data.index))]
min5Data['EpochTime']=[(min5Data.index[k].timestamp()-min5Data.index[0].timestamp())/(60) for k in range(len(min5Data.index))]

#Start loop from maximum period to ensure each EMA function has time to spool up
goodTime=min1Data.EpochTime[min1Data.EpochTime>maxPeriod] 
goodTime=goodTime[:1165] # doing this because we are failing due to slightly less 5min data
data=np.zeros((goodTime.shape[0],25)) # array where the first 21 elements are all the EMAs, then price, enter, exit and time
for t in goodTime: 
    i=np.argmax(t==min1Data.EpochTime) # get index of time vector
    
    print(z)
    # get 1min data for EMA
    tempPrice=min1Data.close.iloc[i] # price at time t
    temp1Data=min1Data.iloc[:i] # historical values at time t

    # get 5 min data for EMA
    ind5=(np.argmax(min5Data.EpochTime>t)) # find index in 5 min data which is > than t
    temp5Price=min5Data.close.iloc[(ind5-1)] # get last 5 minute data price
    temp5Data=min5Data[:(ind5)] # get historical 5 minute values at time t

    # loop through each time step for 1 and 5 minute 
    temp = calcEMA(temp1Data, temp5Data, tempPrice, nVec=nVec) # calculate EMA
    # append dataframe with new row of data
    ema=ema.append(temp)
    price.append(tempPrice)
    time.append(i)

    enterFlag, exitFlag, longBool, shortBool = algoBase(context, temp, tempPrice, longBool, shortBool)

    # take all inputs and load them into the output array
    print(temp.values[0].reshape(1,21))
    data[z,:]=np.concatenate((temp.values[0].reshape(1,21),np.array([tempPrice,enterFlag,exitFlag,z]).reshape(1,4)),axis=1)
    z+=1
ema.index=np.arange(len(ema.index))
ema.insert(0,"time",time)
ema.insert(1,"price",price)
outputPd=pd.DataFrame(data,columns=["fast", "fast_n", "fast_1", "fast_2", 
                  "slow", "slow_n", "slow_1", "fast_slope", "slow_slope",
                  "t_line", "t_line_n", "t_line_1", "t_slope", 
                  "fast_length1", "slow_length_1", "fast_length", "slow_length",
                  "macd", "macd_1", "macd_slope", "macd_9","price","enterFlag","exitFlag","timestep"])    

#     return outputPd


In [ ]:
min1Data.index[0].timestamp()


In [ ]:
plt.plot(outputPd.enterFlag)

In [ ]:
import matplotlib.gridspec as gridspec
# find where entry/exit goes true or false
enters=np.where(outputPd.enterFlag[:-1].values!=outputPd.enterFlag[1:].values,True,False)
exits=np.where(outputPd.exitFlag[:-1].values!=outputPd.exitFlag[1:].values,True,False)
j=0
N=25
M=.001
plotFolder=r'C:\Users\Lane Capital\OneDrive\Desktop\repos\plots\09232022'
# for every enter/exit
for i,flag in enumerate(enters):
    if flag and i>600:
        fig, (ax,ax1,ax2) = plt.subplots(3,1,gridspec_kw={'height_ratios': [2,1,1]},figsize=(10,6))
#         gs = gridspec.GridSpec(2,1,width_ratios=[2,1])
        xlimit=(np.max((i-N,0)),np.min((i+N,len(enters)))) # plot N timesteps before and after flag
        ylimit=(np.min(outputPd.price.values[xlimit[0]:xlimit[1]])*(1-M),np.max(outputPd.price.values[xlimit[0]:xlimit[1]])*(1+M))
        ax.plot(outputPd.timestep.values,outputPd.price.values)
        ax.plot(outputPd.timestep.values,outputPd.fast.values)
        ax.plot(outputPd.timestep.values,outputPd.slow.values)
        ax.set(xlim=xlimit, ylim=ylimit)
        ax.grid(which='major')
        ax.grid(which='minor', linewidth=0.25)
        ax.minorticks_on()
        ax.legend(['price','fast','slow'])
        ax.set_ylabel('USD')
        
        ax1.plot(outputPd.timestep.values,outputPd.macd.values,color='r')
        ax1.plot(outputPd.timestep.values,outputPd.macd_9.values,color='g')
        ylimit_macd=(np.min((np.min(outputPd.macd.values[xlimit[0]:xlimit[1]]),np.min(outputPd.macd_9.values[xlimit[0]:xlimit[1]])))*(1-.1),np.max((np.max(outputPd.macd.values[xlimit[0]:xlimit[1]]),np.max(outputPd.macd_9.values[xlimit[0]:xlimit[1]])))*(1+.1))
        ax1.set(xlim=xlimit,ylim=ylimit_macd)
        ax1.grid(which='major')
        ax1.grid(which='minor', linewidth=0.25)
        ax1.minorticks_on()
        ax1.legend(['macd','Signal Line'])
        ax1.set_ylabel('ND')
        
        
        ax2.plot(outputPd.timestep.values[:-1],enters)
        ax2.plot(outputPd.timestep.values[:-1],exits)
        ax2.set(xlim=xlimit)
        ax2.grid(which='major')
        ax2.grid(which='minor', linewidth=0.25)
        ax2.minorticks_on()
        ax2.legend(['enter flag','exit flag'])
        ax2.set_xlabel('Time Step Since Day Start')
        ax2.set_ylabel('Boolean')
        
        fig.suptitle('Algo Enter/Exit Point')
        fig.savefig(plotFolder+'\plot_'+str(i)+'.png')
        
        j+=1
        if j>=45:
            break

In [ ]:
import matplotlib.gridspec as gridspec
# find where entry/exit goes true or false
enters=np.where(outputPd.enterFlag[:-1].values!=outputPd.enterFlag[1:].values,True,False)
exits=np.where(outputPd.exitFlag[:-1].values!=outputPd.exitFlag[1:].values,True,False)
j=0
N=40
M=.001
inputData=min1Data
times=np.array([min1Data.index[k].timestamp() for k in range(len(min1Data.index))])
plotFolder=r'C:\Users\Lane Capital\OneDrive\Desktop\repos\plots\09112022'
# for every enter/exit
for i,flag in enumerate(enters):
    if flag:
        fig, (ax,ax1,ax2) = plt.subplots(3,1,gridspec_kw={'height_ratios': [2,1,1]},figsize=(10,6))
#         gs = gridspec.GridSpec(2,1,width_ratios=[2,1])
        xlimit=(np.max((i-N,0)),np.min((i+N,len(enters)))) # plot N timesteps before and after flag
        ylimit=(np.min(outputPd.price.values[xlimit[0]:xlimit[1]])*(1-M),np.max(outputPd.price.values[xlimit[0]:xlimit[1]])*(1+M))
        
        #define width of candlestick elements
        width = 0.5
        width2 = 0.05

        #define up and down inputData
        up = inputData[inputData.close>=inputData.open]
        down = inputData[inputData.close<inputData.open]
        
        upTimes=np.array([(up.index[k].timestamp()-inputData.index[0].timestamp())/60 for k in range(len(up.index))])
        downTimes=np.array([(down.index[k].timestamp()-inputData.index[0].timestamp())/60  for k in range(len(down.index))])
        
        #define colors to use
        col1 = 'green'
        col2 = 'red'

        #plot up inputData
        ax.bar(upTimes,up.close-up.open,width,bottom=up.open,color=col1)
        ax.bar(upTimes,up.high-up.close,width2,bottom=up.close,color=col1)
        ax.bar(upTimes,up.low-up.open,width2,bottom=up.open,color=col1)

        #plot down inputData
        ax.bar(downTimes,down.close-down.open,width,bottom=down.open,color=col2)
        ax.bar(downTimes,down.high-down.open,width2,bottom=down.open,color=col2)
        ax.bar(downTimes,down.low-down.close,width2,bottom=down.close,color=col2)    
    
#         ax.plot(outputPd.timestep.values,outputPd.price.values)
        ax.plot(outputPd.timestep.values,outputPd.fast.values)
        ax.plot(outputPd.timestep.values,outputPd.slow.values)
        ax.set(xlim=xlimit, ylim=ylimit)
        ax.grid(which='major')
        ax.grid(which='minor', linewidth=0.25)
        ax.minorticks_on()
        ax.legend(['price','fast','slow'])
        ax.set_ylabel('USD')
        
        ax1.plot(outputPd.timestep.values,outputPd.macd.values,color='r')
        ax1.plot(outputPd.timestep.values,outputPd.macd_9.values,color='g')
        ylimit_macd=(np.max((np.min(outputPd.macd.values[xlimit[0]:xlimit[1]]),np.min(outputPd.macd_9.values[xlimit[0]:xlimit[1]])))*(1-.05),np.min((np.max(outputPd.macd.values[xlimit[0]:xlimit[1]]),np.max(outputPd.macd_9.values[xlimit[0]:xlimit[1]])))*(1+.05))
        ax1.set(xlim=xlimit,ylim=ylimit_macd)
        ax1.grid(which='major')
        ax1.grid(which='minor', linewidth=0.25)
        ax1.minorticks_on()
        ax1.legend(['macd','macd_9'])
        ax1.set_ylabel('ND')
        
        
        ax2.plot(outputPd.timestep.values[:-1],enters)
        ax2.plot(outputPd.timestep.values[:-1],exits)
        ax2.set(xlim=xlimit)
        ax2.grid(which='major')
        ax2.grid(which='minor', linewidth=0.25)
        ax2.minorticks_on()
        ax2.legend(['enter flag','exit flag'])
        ax2.set_xlabel('Time Step Since Day Start')
        ax2.set_ylabel('Boolean')
        
        fig.suptitle('Algo Enter/Exit Point')
#         fig.savefig(plotFolder+'\plot_'+str(i)+'.png')

        j+=1
        if j>=5:
            break

In [ ]:
ylimit=(np.max(outputPd.price.values[xlimit[0]:xlimit[1]])*1.05,np.min(outputPd.price.values[xlimit[0]:xlimit[1]])*.95)

In [ ]:
len(min1Data.index)

In [ ]:
upTimes/60

In [ ]:
ylimit_macd=(np.max((np.min(outputPd.macd.values[xlimit[0]:xlimit[1]]),np.min(outputPd.macd_9.values[xlimit[0]:xlimit[1]]))),np.min((np.max(outputPd.macd.values[xlimit[0]:xlimit[1]]),np.max(outputPd.macd_9.values[xlimit[0]:xlimit[1]]))))

In [ ]:
xlimit=(150,300)

(np.min((np.min(outputPd.macd.values[xlimit[0]:xlimit[1]]),np.min(outputPd.macd_9.values[xlimit[0]:xlimit[1]])))*(1+.05),np.max((np.max(outputPd.macd.values[xlimit[0]:xlimit[1]]),np.max(outputPd.macd_9.values[xlimit[0]:xlimit[1]])))*(1-.05))

In [ ]:
np.max(outputPd.macd_9.values[xlimit[0]:xlimit[1]])